## DATA LOAD

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.linear_model import Ridge
import pywt
from sklearn.preprocessing import MinMaxScaler

# 최대 줄 수 설정
#pd.set_option('display.max_rows', 500)
# 최대 열 수 설정
#pd.set_option('display.max_columns', 500)

In [6]:
data_x = pd.read_csv("train_x_1.csv")
data_y = pd.read_csv("train_y_1.csv")
del data_y['ID']

In [3]:
test_x = pd.read_csv("test_x_1.csv")


In [8]:
data_x.head()

,ID,0,1,2,3,4,5,6,7,8,...,658,659,660,661,662,663,664,665,666,667
0,0,5519,4651,3787,3619,3714,3936,4137,3874,3414,...,1171,1356,1430,1455,1538,1656,1802,2017,2306,2568
1,0,5511,4645,3769,3640,3707,3928,4138,3867,3433,...,1174,1354,1429,1462,1539,1664,1812,2011,2302,2571
2,0,5490,4626,3769,3607,3692,3921,4111,3847,3398,...,1169,1351,1424,1453,1539,1657,1805,2013,2296,2550
3,0,5487,4629,3766,3610,3693,3916,4122,3850,3415,...,1166,1353,1431,1455,1537,1650,1784,2001,2306,2574
4,0,5464,4616,3758,3595,3708,3942,4133,3848,3398,...,1167,1351,1427,1454,1532,1652,1807,2010,2293,2566


## DATA PROPRECESSING

+ 공정 데이터로 ID 별로 1698개의 시계열 데이터를 가지고 있고, 667개의 변수를 가지고 있다.
+ ID가 400개 밖에 안되기 때문에, 대표적이 LARGE P , SMALL N 문제라고 볼 수 있다

### 기초 통계량 

In [50]:
def basic(data_x):
    mean_df = data_x.groupby(by= 'ID').mean()
    std_df = data_x.groupby(by= 'ID').std()
    max_df = data_x.groupby(by= 'ID').max()
    min_df = data_x.groupby(by= 'ID').min()
    q1_df = data_x.groupby(by= 'ID').quantile(.25)
    q2_df = data_x.groupby(by= 'ID').quantile(.50)
    q3_df = data_x.groupby(by= 'ID').quantile(.75)
    
    basic_df = pd.concat([mean_df,std_df,max_df,min_df,q1_df,q2_df,q3_df],axis=1)
    return basic_df

basic_df = basic(data_x)

In [146]:
basic_df = pd.concat([mean_df,std_df,max_df,min_df,q1_df,q2_df,q3_df],axis=1)
basic_df.to_csv("basic.csv",index =False)

In [51]:
t_basic_df = basic(test_x)

In [52]:
#TEST SET
t_basic_df.to_csv("t_basic.csv",index =False)

In [55]:
t_basic_df.shape

(101, 4676)

### 1. 단위별 평균 값
- lm 에서는 100개씩 lenth = 5일때 성능이 제일 좋음
- Lasso는 ?
- Ridge는 가장 많이 나눌 때 성능ㅇ ㅣ좋음

In [223]:
split_data = []
def cutting(start,lenth):
    tem_index = []
    if start == 1600:
        lenth = 97
    
    for num in range(404):
        tem = [i for i in range(1697*num+start, 1697*num+start+lenth)]
        tem_index.extend(tem)
        
    return tem_index
    

for N in range(0,17):
    tem = cutting(N*100,5)
    split_data.append(tem)
    
# 마지막 



In [216]:
data_house = []
for idx in split_data:
    tem_data = data_x.iloc[idx]
    tem_mean = tem_data.groupby(by= 'ID').mean()
    #tem_std = tem_data.groupby(by= 'ID').std()
    #tem_max = tem_data.groupby(by= 'ID').max()
    #tem_min = tem_data.groupby(by= 'ID').min()
    data_house.append(tem_mean)
    #data_house.append(tem_std)
    #data_house.append(tem_max)
    #data_house.append(tem_min)

In [217]:
basic_df = pd.concat([data for data in data_house],axis=1)

In [218]:
basic_df.shape

(404, 11356)

In [222]:
#basic_df.to_csv("r_data_div_100_len5.csv",index =False)

In [ ]:
#test

In [71]:
t_split_data = []
def cutting(start,lenth):
    tem_index = []
    if start == 1600:
        lenth = 97
    
    for num in range(101):
        tem = [i for i in range(1697*num+start, 1697*num+start+lenth)]
        tem_index.extend(tem)
        
    return tem_index
    

for N in range(0,17):
    tem = cutting(N*100,5)
    t_split_data.append(tem)

In [72]:
t_data_house = []
for idx in t_split_data:
    tem_data = test_x.iloc[idx]
    tem_mean = tem_data.groupby(by= 'ID').mean()
    t_data_house.append(tem_mean)

In [73]:
t_basic_df = pd.concat([data for data in t_data_house],axis=1)
t_basic_df.shape

(101, 11356)

In [74]:
t_basic_df.to_csv("t_div_100.csv",index =False)

### 2.  max es반영 (max 2,3,4 번째 데이터 더 반영)(효과X)

In [134]:
import tqdm as tqdm
max_list = []
for i in tqdm.tqdm(range(1,10)):
    tem = 0.9 * 0.1**i 
    tem_data = data_x.groupby(by= 'ID').quantile(tem)
    max_list.append(tem_data)
len(max_list)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [08:22<00:00, 55.81s/it]


9

In [136]:
maxes_data = pd.concat([x for x in max_list],axis=1)
maxes_data.shape

(404, 6012)

### 3. q4~ W1~W6 변수 (HANDS ON)

In [67]:
def hands(data_x):
    q4_df = data_x.groupby(by= 'ID').quantile(.85)
    q5_df = data_x.groupby(by= 'ID').quantile(.91)

    q6_df = data_x.groupby(by= 'ID').quantile(.95)
    q7_df = data_x.groupby(by= 'ID').quantile(.98)

    q8_df = data_x.groupby(by= 'ID').quantile(.99)
    q9_df = data_x.groupby(by= 'ID').quantile(.995)
    q10_df = data_x.groupby(by= 'ID').quantile(.999)
    
    w1 = data_x.groupby(by= 'ID').quantile(.9992)
    w2 = data_x.groupby(by= 'ID').quantile(.9995)
    w3 = data_x.groupby(by= 'ID').quantile(.9998)
    
    w4 = data_x.groupby(by= 'ID').quantile(.9999)
    w5 = data_x.groupby(by= 'ID').quantile(.99993)
    w6 = data_x.groupby(by= 'ID').quantile(.99998)

    tem = pd.concat([q4_df,q5_df,q6_df,q7_df,q8_df,q9_df,q10_df,w1,w2,w3,w4,w5,w6],axis = 1)
    
    return tem

In [ ]:
hands_on = pd.concat([q4_df,q5_df,q6_df,q7_df,q8_df,q9_df,q10_df,w1,w2,w3,w4,w5,w6],axis = 1)
hands_on.to_csv("hands_w6.csv",index =False)

In [68]:
t_hands_on = hands(test_x)
t_hands_on.shape

(101, 8684)

In [69]:
t_hands_on.to_csv("t_hands_w6.csv",index =False)

# K-fold

In [138]:
from sklearn.model_selection import KFold

def k_fold(basic_df,model):
    split_num =10
    kfold = KFold(n_splits=split_num, shuffle=True, random_state=10)
    score = []
    for train_index,test_index in kfold.split(basic_df):
        train_x,valid_x = basic_df.iloc[train_index],basic_df.iloc[test_index]
        train_y,valid_y = data_y.iloc[train_index],data_y.iloc[test_index]
        
        #model select
        if model == "lm":
            model_use = LinearRegression(fit_intercept=True, normalize=True, n_jobs=None)
        
        elif model == "lasso":
            model_use = linear_model.Lasso(alpha=0.1)
        elif model == "ridge":
            model_use = Ridge(alpha=0.1)
        else:
            print("모델이 존재하지 않습니다.")
            break
        model_use.fit(train_x, train_y)
    
        valid_pred = model_use.predict(valid_x)
        tem = mean_squared_error(valid_pred,valid_y)
    
        score.append(tem)
        print("=============================check=================================")
        
    total_score = np.mean(score)
    print("%s 모델 K-fold 결과 : %f "%(model,total_score))
    return
        

In [143]:
maxes_data.to_csv("maxes_data.csv",index =False)

In [141]:
k_fold(maxes_data,"ridge")
# div100 len5 -> 0.0589 -> 기존과 결합했을 때 가장 준수한 성능 
# div25 len5 -> 0.0573
# div 1- len5 -> 0.0565

=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
=============================check=================================
ridge 모델 K-fold 결과 : 0.212473 


# ===========================================================

### 1.2 Wavelet

In [148]:
test = data_x['1']

In [149]:
cA, cD = pywt.dwt(test, 'db1')

In [150]:
cA.shape

(342794,)

In [4]:
cA, cD = pywt.dwt(data_x, 'db1')
wave = pd.concat([pd.DataFrame(cA),pd.DataFrame(cD)],axis = 1)
wave['ID'] = data_x['ID']

mean_df = wave.groupby(by= 'ID').mean()
std_df = wave.groupby(by= 'ID').std()
max_df = wave.groupby(by= 'ID').max()
min_df = wave.groupby(by= 'ID').min()

wave_df = pd.concat([mean_df],axis = 1)
wave_df.to_csv("wave_df_meanonly.csv",index=False)

In [5]:
wave_df = pd.concat([mean_df,std_df,max_df,min_df],axis = 1)
wave_df.to_csv("wave_df.csv",index=False)

In [6]:
cA, cD = pywt.dwt(test_x, 'db1')
wave = pd.concat([pd.DataFrame(cA),pd.DataFrame(cD)],axis = 1)
wave['ID'] = test_x['ID']

mean_df = wave.groupby(by= 'ID').mean()
std_df = wave.groupby(by= 'ID').std()
max_df = wave.groupby(by= 'ID').max()
min_df = wave.groupby(by= 'ID').min()

t_wave_df = pd.concat([mean_df],axis = 1)
t_wave_df.to_csv("t_wave_df_meanonly.csv",index=False)

In [7]:
t_wave_df = pd.concat([mean_df,std_df,max_df,min_df],axis = 1)
t_wave_df.to_csv("t_wave_df.csv",index=False)

## 시계열을 자체를 PCA 해보기 ( 효과 X)

In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=50)
tem = pca.fit_transform(data_x.T)

pca_df = pd.DataFrame(tem)


In [8]:
pca_df.T.shape

(10, 669)

In [28]:
from tqdm import tqdm
pca_list =[]
for num in tqdm(range(404)):
    tem_data = data_x[data_x['ID'] == num] #ID별 데이터
    
    pca = PCA(n_components=50)
    tem = pca.fit_transform(tem_data.T) 
    
    tem2 = tem.T # 50 * 662
    tem2 = tem2.reshape(-1)
    pca_list.append(tem2)
    

100%|████████████████████████████████████████████████████████████████████████████████| 404/404 [00:18<00:00, 21.33it/s]


In [31]:
pca_df = np.concatenate([i for i in pca_list],axis = 0)

In [42]:
a = np.array(pca_list)
pca_df =  pd.DataFrame(a)
pca_df.shape

(404, 33450)

In [47]:
pca_df.to_csv("pca_df.csv",index =False)

## 구간별로 나눠서 표준편차 구하기 (효과 X)

In [4]:
split_data = []
def cutting(start,lenth):
    tem_index = []
    if start == 1600:
        lenth = 97
    
    for num in range(404):
        tem = [i for i in range(1697*num+start, 1697*num+start+lenth)]
        tem_index.extend(tem)
        
    return tem_index
    

for N in range(0,17):
    tem = cutting(N*100,100)
    split_data.append(tem)
    
# 마지막 



In [5]:
data_house = []
for idx in split_data:
    tem_data = data_x.iloc[idx]
    #tem_mean = tem_data.groupby(by= 'ID').mean()
    tem_std = tem_data.groupby(by= 'ID').std()
    #tem_max = tem_data.groupby(by= 'ID').max()
    #tem_min = tem_data.groupby(by= 'ID').min()
    data_house.append(tem_std)
    #data_house.append(tem_std)
    #data_house.append(tem_max)
    #data_house.append(tem_min)

In [6]:
std_df = pd.concat([data for data in data_house],axis=1)
std_df.shape

(404, 11356)

In [8]:
std_df.to_csv("std_div.csv",index = False)

## BASIC 2 - Quantile 값들 ( Q1,Q2 사이도 추가해보기 ) (효과)

In [9]:
def basic(data_x):
    mean_df = data_x.groupby(by= 'ID').mean()
    std_df = data_x.groupby(by= 'ID').std()
    max_df = data_x.groupby(by= 'ID').max()
    min_df = data_x.groupby(by= 'ID').min()
    q1_df = data_x.groupby(by= 'ID').quantile(.1)
    q2_df = data_x.groupby(by= 'ID').quantile(.5)
    q3_df = data_x.groupby(by= 'ID').quantile(.10)
    q4_df = data_x.groupby(by= 'ID').quantile(.20)

    q5_df = data_x.groupby(by= 'ID').quantile(.25)
    q6_df = data_x.groupby(by= 'ID').quantile(.40)
    q7_df = data_x.groupby(by= 'ID').quantile(.50)
    q8_df = data_x.groupby(by= 'ID').quantile(.60)
    q9_df = data_x.groupby(by= 'ID').quantile(.75)
    
    basic_df = pd.concat([mean_df,std_df,max_df,min_df,q1_df,q2_df,q3_df,q4_df,q5_df,q6_df,q7_df,q8_df,q9_df],axis=1)
    return basic_df

basic_df = basic(data_x)

In [10]:
basic_df.to_csv("basic2_df.csv",index = False)

## 최근 시계열 짤라서 표준편차 구하기 (효과X) 

In [41]:
split_data = []
def cutting(start,lenth):
    tem_index = []
    if start == 1600:
        lenth = 97
    
    for num in range(404):
        tem = [i for i in range(1697*num+start, 1697*num+start+lenth)]
        tem_index.extend(tem)
        
    return tem_index
    

for N in range(0,17):
    tem = cutting(N*100,5)
    split_data.append(tem)
    
# 마지막 



In [42]:
tem_index=[]
for num in range(0,404):
    tem = [i for i in range(1697*num, 1697*num+1600)]
    tem_index.extend(tem)

In [43]:
data_x.drop(tem_index,inplace = True)

In [44]:
half_std = data_x.groupby(by = 'ID').std()

In [45]:
half_std.to_csv("half_std.csv",index = False)